In [125]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [126]:
example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [127]:
def mix(num1,num2):
    return num1 ^ num2

def prune(num):
    return num % 16777216

def find_next_secret_number(secret_num):
    step1 = prune(mix(secret_num,secret_num*64))
    step2 = prune(mix(step1, step1//32))
    step3 = prune(mix(step2, step2*2048))
    return step3

def find_xth_secret_number(secret_num, x):
    for _ in range(x):
        secret_num = find_next_secret_number(secret_num)
    return secret_num


In [128]:
# initial_secret_numbers = input.split('\n')

# total = 0
# for init_secret_num in initial_secret_numbers:
#     total += find_xth_secret_number(int(init_secret_num), 2000)
# total

# Part 2

In [129]:
class Monkey():
    def __init__(self, initial_secret_number):
        # Find prices and changes for first 2000 secret numbers
        self.prices = [int(str(initial_secret_number)[-1])]
        self.changes = [None]

        # Make hashmap of sequences that appear in this monkey to the number of bananas
        self.seqs = {}

        cur_secret_num = int(initial_secret_number)
        for i in range(2000):
            cur_secret_num = find_next_secret_number(cur_secret_num)
            banana_price = int(str(cur_secret_num)[-1])
            self.prices.append(banana_price)
            if i == 0:
                continue
            change = self.prices[i] - self.prices[i-1]
            self.changes.append(change)
            if i > 3:
                cur_seq = (self.changes[i-3],self.changes[i-2],self.changes[i-1],self.changes[i])
                if cur_seq not in self.seqs:
                    self.seqs[cur_seq] = self.prices[i]



def score_seq(seq, monkeys):
    '''
    Takes a sequences of changes and returns the total number of bananas returned
    when exchanged across all of the monkeys
    '''
    total = 0
    for monkey in monkeys:
        if seq in monkey.seqs:
            total += monkey.seqs[seq]
    return total


In [133]:
from tqdm import tqdm

initial_secret_numbers = input.split('\n')

# Order of buyers doesn't matter, so store each monkey in a set
monkeys = set()

# Keep track of all possible sequences that appear
all_seqs = set()

# Initialise monkeys
for monkey in tqdm(initial_secret_numbers, desc='Generating monkeys...'):
    cur_monkey = Monkey(monkey)
    monkeys.add(cur_monkey)
    all_seqs = all_seqs.union(set(cur_monkey.seqs.keys()))


most_bananas = 0
for seq in tqdm(all_seqs, desc='Testing sequences...'):
    most_bananas = max(most_bananas, score_seq(seq, monkeys))
most_bananas

Testing sequences...: 100%|██████████| 40951/40951 [00:16<00:00, 2446.10it/s]


1791